In [ ]:
import sys
import torchvision
import numpy as np
from envs.navigation.State import State
from envs.navigation.DiscreteStateSpace import DiscreteStateSpace, XYClassDistribution
from envs.navigation.Dynamics import XYDynamics
from envs.navigation.StateFeature import FeatureStateIndicator, FeatureStateIndicatorOneHot, \
                                        FeatureClassIndicator, FeatureClassIndicatorOneHot, \
                                        FeatureClassImage, FeatureClassImageSampler
from envs.navigation.StateReward import RewardStateScalar
import rl.planning as Plan
import rl.policy as Policy

sys.path.append("./NavigationGridViews/")
from NavigationGridViews.navgridviews.Raster import Raster as NavGridView

from matplotlib import cm as cm, pyplot as plt, colors as mplotcolors
%load_ext autoreload
%autoreload 2

In [ ]:
cifar = torchvision.datasets.CIFAR100(root="./", download=True)
X, y = cifar.data, np.asarray(cifar.targets)
classes = np.unique(y)
n_classes = len(classes)

print(X.shape, y.shape)

def sample_image(X, y, y_q):
    x_q = X[y == y_q]
    return x_q[np.random.randint(0, len(x_q))]

In [ ]:
cifar.classes.index("house"), cifar.classes.index("skyscraper"), cifar.classes.index("road")

## Gridworld 5x5

**sanity check**

In [ ]:
np.random.seed(5)
# Specify state space
S = DiscreteStateSpace(5,5)
goal = (4,4)

# Attach classes
S.sample_and_attach_classes([37, 76, 68], [0.2, 0.3, 0.5])
S.override_classes_by_loc([goal], [68])
# Attach features
S.attach_features(FeatureClassImageSampler(S, lambda x: sample_image(X, y, x)))
PHI_gridded = S.features(gridded=True)
PHI_gridded_vis = np.flip(PHI_gridded, 2)
# Attach rewards
S.attach_rewards(RewardStateScalar(S, loc_to_reward_dict={goal: 1}, class_id_to_reward_dict={
    37: 0, 76: 0, 68: 0}, default=0))
R = S.rewards()
# Specify dynamics
T = XYDynamics(S, slip_prob=0.)
# Set goal
S.set_terminal_status_by_loc([goal])
# Value Iteration
VI = Plan.ValueIteration(S, R, T, verbose=True)
# VI.q_value_list(S[23], False)

In [ ]:
log_Pi, V, Q, iterno = VI.run(100, Policy.Greedy, verbose=True, debug=False, eps=1e-6)

In [ ]:
fig = plt.figure(figsize=(14,14))
state_ann_sz = 14
state_ann_color = "white"
plt.subplot(2,2,1)
p = NavGridView(PHI_gridded_vis, ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    S.class_ids, fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Navigation World (class id)")
# plt.colorbar(p.im)
p.ax.invert_yaxis()

plt.subplot(2,2,2)
p = NavGridView(R[..., np.newaxis, np.newaxis, np.newaxis], ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    R, fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Reward")
# plt.colorbar(p.im)
p.ax.invert_yaxis()

plt.subplot(2,2,3)
p = NavGridView(PHI_gridded_vis, ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    V.numpy().round(2), fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Navigation World (values)")
p.ax.invert_yaxis()

plt.subplot(2,2,4)
p = NavGridView(S._organize_to_grid(V)[..., np.newaxis, np.newaxis, np.newaxis], 
                ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    V.numpy().round(2), fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Value")
p.ax.invert_yaxis()

**Trying different reward function**

In [ ]:
class_ids = XYClassDistribution({"r": 68, "h": 37, "s": 76}, 
                                    ["rrrrr",
                                     "rhhhr",
                                     "rhhhr",
                                     "rhhhr",
                                     "rrrrr"])().flatten()
S.attach_classes(class_ids)
S.override_classes_by_loc([goal], [68])
S.attach_features(FeatureClassImageSampler(S, lambda x: sample_image(X, y, x)))
PHI_gridded = S.features(gridded=True)
PHI_gridded_vis = np.flip(PHI_gridded, 2)

In [ ]:
S.attach_rewards(RewardStateScalar(S, loc_to_reward_dict={goal: 0}, 
                                   class_id_to_reward_dict={37: -1., 76: -1., 68: -0.1}, default=0))
R = S.rewards()

In [ ]:
VI = Plan.ValueIteration(S, R, T, verbose=True)
log_Pi, V, Q, iterno = VI.run(100, Policy.Greedy, verbose=True, debug=False, eps=1e-6)

In [ ]:
fig = plt.figure(figsize=(14,14))
state_ann_sz = 14
state_ann_color = "white"
plt.subplot(2,2,1)
p = NavGridView(PHI_gridded_vis, ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    S.class_ids, fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Navigation World (class id)")
# plt.colorbar(p.im)
p.ax.invert_yaxis()

plt.subplot(2,2,2)
p = NavGridView(R[..., np.newaxis, np.newaxis, np.newaxis], ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    R, fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Reward")
# plt.colorbar(p.im)
p.ax.invert_yaxis()

plt.subplot(2,2,3)
p = NavGridView(PHI_gridded_vis, ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    V.numpy().round(2), fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Navigation World (values)")
p.ax.invert_yaxis()

plt.subplot(2,2,4)
p = NavGridView(S._organize_to_grid(V)[..., np.newaxis, np.newaxis, np.newaxis], 
                ax=plt.gca()).render().ticks(minor=False).show_cell_text(
    V.numpy().round(2), fontsize=state_ann_sz, color_cb=lambda x: state_ann_color).title("Value")
p.ax.invert_yaxis()